In [20]:
# PTT title generator from content base on seq2seq model.
import pandas as pd
import time
import csv
import os
import types 

#Initialize for files name and path.
base_dir = 'C:/Users/User/Raw data/PTT'
totalboard_name = 'TJ-BG-AT-SX-CC-MV'
board_name = ['Tech_job','Boy-Girl','AllTogether','sex','C_Chat','movie']
board_dict_index = {'Tech_job':0,'Boy-Girl':1,'AllTogether':2,'sex':3,'C_Chat':4,'movie':5}
max_word_length = 256
min_word_length = 100
max_title_length = 20
label_newnum = len(board_name)

new_content_dir = []
totalboard_dir = os.path.join(base_dir, totalboard_name)
if not os.path.exists(totalboard_dir):
    os.makedirs(totalboard_dir)
for name_ind in range(len(board_name)):
    # New data path
    new_content_dir.append(os.path.join(totalboard_dir, board_name[name_ind]+'_content'))
    if not os.path.exists(new_content_dir[name_ind]):
        os.makedirs(new_content_dir[name_ind])

from sklearn.cross_validation import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import jieba

# Import dictionary fron jieba and PTT.
jieba.set_dictionary('C:/Users/User/Anaconda3/Lib/site-packages/jieba/dict.txt.big')
jieba.load_userdict('C:/Users/User/Anaconda3/Lib/site-packages/jieba/userdict.txt')

text_content = []
text_label = []
text_title = []

GO_word = '<GO>' #解碼器端的句子起始標識符。
EOS_word = '<EOS>' #解碼器端的句子結束標識符。
PAD_word = '<PAD>' #補全字符。
UNK_word = '<UNK>' #低頻詞或者一些未遇到過的詞等。

# Load PTT content and information.
#dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '-test.csv'))
dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '.csv'))
len_total = len(dfs_total)

for dfs_index in range(len_total):
    read_index = int(dfs_total.iloc[dfs_index, 0])
    type_name = str(dfs_total.iloc[dfs_index, 1])
    word_length = int(dfs_total.iloc[dfs_index, 5])
    title_name = str(dfs_total.iloc[dfs_index, 9])
    this_board_name = str(dfs_total.iloc[dfs_index, 10])
    
    # Add start and end character.
    text = GO_word + ' '
    with open(os.path.join(new_content_dir[board_dict_index[this_board_name]], str(read_index) + '.csv'),
                'r', encoding = 'utf-8-sig') as file:
        csvCursor = csv.reader(file)
        for rows in csvCursor:
            for row in rows:
                # Read content and remove empty.
                text = text + row
        else:
            text = text + ' ' + EOS_word
    
    # Split titles to words by Jieba.
    title_name.encode('utf-8-sig')
    title_jieba = jieba.cut(title_name, cut_all=False)
    title_name = GO_word
    word_length = 0
    for word in title_jieba:
        word_length += 1
        title_name = title_name + ' ' + word
    else:
        title_name = title_name + ' ' + EOS_word

    text_content.append(text)
    text_label.append(board_dict_index[this_board_name])
    text_title.append(title_name)
    file.close()

# Build vocabulary and convert content to sequence by Keras tool.
words_limit = 60000
tokenizer = Tokenizer(num_words=words_limit, 
                      filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n　，。！：；、？﹝﹞「」『』（）｛｝［］【】《》“”‘’＼｜〝〞‵′＋－＊／＝≦≧＿＠＃＄％︿＆～§◎．※ㄧ↔│○●☆★◇◆□■▽▼△▲㊣⊙⊕ˍ…﹌﹋﹎﹍﹉﹊‥–↑↓←→↖↗↙↘∥∕℅≒≡∩∪∞￣＿◤◥◣◢∵∴〒⊥∠⊿┼┴┬┤├▔─│▕┌┐└┘╭╮╰╯═╞╪╡╔╦╗╠╬╣╚╩╝╒╤╕╘╧╛╓╥╖╟╫╢╙╨╜║▓╱╲╳▁▂▄▅▆▇█▏▎▍▌▋▊▉▁▔', 
                      lower=False,
                      split=" ")
                      #oov_token=UNK_word)
tokenizer.fit_on_texts(text_content+text_title)
vocab = tokenizer.word_index
vocab_counts = tokenizer.word_counts
vocab_docs = tokenizer.word_docs

print(len(vocab))
anti_vocab = {}
vocab_keys = list(vocab.keys())
for vocab_word in vocab:
    vocab_index = vocab[vocab_word]
    anti_vocab[vocab_index] = vocab_keys[vocab_index-1]
    if vocab_index % 1000 == 0 or vocab_index < 20:
        print(vocab[vocab_word], vocab_word, anti_vocab[vocab_index], vocab_counts[vocab_word], vocab_docs[vocab_word])

    
from keras.utils import np_utils

x_train, x_test, y_train, y_test = train_test_split(text_content, text_title, test_size=0.2, random_state=9487)

# 将每个词用词典中的数值代替
x_train_word_ids = tokenizer.texts_to_sequences(x_train)
x_test_word_ids = tokenizer.texts_to_sequences(x_test)
y_train_word_ids = tokenizer.texts_to_sequences(y_train)
y_test_word_ids = tokenizer.texts_to_sequences(y_test)
# 序列模式
x_train = pad_sequences(x_train_word_ids, maxlen=max_word_length+2, padding='post', truncating='post')
x_test = pad_sequences(x_test_word_ids, maxlen=max_word_length+2, padding='post', truncating='post')
y_train = pad_sequences(y_train_word_ids, maxlen=max_title_length+2, padding='post', truncating='post')
y_test = pad_sequences(y_test_word_ids, maxlen=max_title_length+2, padding='post', truncating='post')
# 序列長度
x_train_size = [len(word_ids) for word_ids in x_train_word_ids]
x_test_size = [len(word_ids) for word_ids in x_test_word_ids]
y_train_size = [len(word_ids) for word_ids in y_train_word_ids]
y_test_size = [len(word_ids) for word_ids in y_test_word_ids]

Building prefix dict from C:\Users\User\Anaconda3\Lib\site-packages\jieba\dict.txt.big ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ufa6ae29b0cbce8b45e006c7fa30eaaf8.cache
Loading model cost 1.077 seconds.
Prefix dict has been built succesfully.


199446
1 的 的 479528 108422
2 我 我 187622 60294
3 <GO> <GO> 172800 172800
4 <EOS> <EOS> 172800 172800
5 是 是 157302 68972
6 了 了 129931 60543
7 有 有 108041 59614
8 在 在 97066 54361
9 也 也 86587 48759
10 都 都 75404 45560
11 就 就 71400 42410
12 不 不 66238 42797
13 很 很 54725 34820
14 會 會 54210 36201
15 他 他 49560 20170
16 跟 跟 48539 31439
17 看 看 48476 30452
18 你 你 48168 21707
19 想 想 47770 34168
1000 跳 跳 1246 1022
2000 整合 整合 607 489
3000 往往 往往 366 345
4000 007 007 255 156
5000 處於 處於 192 188
6000 一般來說 一般來說 154 150
7000 何 何 125 101
8000 有鬼 有鬼 105 100
9000 備 備 89 89
10000 半年前 半年前 78 78
11000 掉下來 掉下來 68 66
12000 托爾 托爾 61 46
13000 傻子 傻子 54 51
14000 活得 活得 49 48
15000 寫給 寫給 44 41
16000 希斯 希斯 41 31
17000 毆打 毆打 37 33
18000 編制 編制 34 29
19000 激怒 激怒 32 30
20000 神秘性 神秘性 30 5
21000 睡意 睡意 27 27
22000 真情 真情 26 24
23000 寫些 寫些 24 24
24000 野蠻女友 野蠻女友 23 14
25000 20th 20th 21 17
26000 世家 世家 20 20
27000 天降 天降 19 18
28000 人卡 人卡 18 13
29000 決定權 決定權 17 17
30000 始於 始於 16 15
31000 初號機 初號機 16 10
32000 早睡早起 早睡早起 15 15
33000 記號 記號

In [55]:
################
# Build seq2seq model
################
def build_model_in():
    ######## model input tensor ########
    is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    input_seq_len = tf.placeholder(tf.int32, (None,), name='input_seq_len')
    target_seq_len = tf.placeholder(tf.int32, (None,), name='target_seq_len')
    limit_target_seq_len = tf.placeholder(tf.int32, (), name='limit_target_seq_len')
    return is_training, inputs, targets, input_seq_len, target_seq_len, limit_target_seq_len
    
def build_model(is_training, inputs, targets, input_seq_len, target_seq_len, limit_target_seq_len):
    max_target_seq_len = tf.reduce_max(target_seq_len, name='max_target_seq_len')
    validation_batch_size = tf.shape(targets)[0]
    start_tokens = tf.tile(tf.constant([vocab[GO_word]], dtype=tf.int32),
                               [validation_batch_size], name='start_tokens')
    
    ######## Embedding for encoder and decoder ########
    embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size]))
    encoder_embed_input = tf.nn.embedding_lookup(embeddings, inputs)
    decoder_embed_input = tf.nn.embedding_lookup(embeddings, targets)
    
    ######## Encoder by LSTM ########
    # LSTM cell
    def get_lstm_cell(rnn_size, layer):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, 
                                            initializer=tf.random_uniform_initializer(-0.1, 0.1)) 
        tf.contrib.layers.l2_regularizer(l2_reg_constant)
        if is_training is not None and lstm_dropout < 1.0:
            if layer == 0:
                lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, 
                                                          input_keep_prob=lstm_dropout, 
                                                          output_keep_prob=lstm_dropout)
            else:
                lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, 
                                                          output_keep_prob=lstm_dropout)
        return lstm_cell

    # Struct encoder by LSTM bi-directional multi-layer
    encoder_f_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size, layer) for layer in range(num_layers)])
    encoder_b_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size, layer) for layer in range(num_layers)])
    encoder_bi_outputs, encoder_bi_state = tf.nn.bidirectional_dynamic_rnn(encoder_f_cell, 
                                                                           encoder_b_cell, 
                                                                           encoder_embed_input, 
                                                                           sequence_length=input_seq_len, 
                                                                           dtype=tf.float32)
    encoder_outputs = tf.concat((encoder_bi_outputs[0], encoder_bi_outputs[1]), 2)
    encoder_state_c = [tf.concat((state[0].c, state[1].c), 1) for state in encoder_bi_state]
    encoder_state_h = [tf.concat((state[0].h, state[1].h), 1) for state in encoder_bi_state]
    encoder_state = tf.contrib.rnn.LSTMStateTuple(c=encoder_state_c, h=encoder_state_h)
    
    ######## Decoder by LSTM ########
    # Bahdanau attention
    def get_decoder_cell(rnn_size, layer, attn_layers, encoder_outputs, input_seq_len, slot_size):
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(rnn_size*2, encoder_outputs,
                                                                   memory_sequence_length=input_seq_len)
        attn_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(get_lstm_cell(rnn_size*2, layer), 
                                                               attention_mechanism, 
                                                               attention_size=attn_layers)
        #decoder_cell = tf.contrib.rnn.OutputProjectionWrapper(attn_cell, slot_size)
        return attn_cell
    
    # Struct LSTM cell in Decoder
    decoder_cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_size, layer, num_layers, encoder_outputs, input_seq_len, slot_size) for layer in range(num_layers)])
    
    # Output fully-connecting layer
    output_layer = Dense(vocab_size, 
                         kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    tf.contrib.layers.l2_regularizer(l2_reg_constant)
    
    # Struct training decoder
    with tf.variable_scope('decode'):
        training_helper = tf.contrib.seq2seq.TrainingHelper(decoder_embed_input,
                                                            target_seq_len)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, training_helper,
                                                           decoder_cell.zero_state(dtype=tf.float32, batch_size=batch_size), 
                                                           output_layer) 
        training_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                       impute_finished=True,
                                                                       maximum_iterations=max_target_seq_len)
    # Struct predicting decoder (Share variable with training)
    with tf.variable_scope('decode', reuse=True):
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                     start_tokens, 
                                                                     vocab[EOS_word])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, predicting_helper,
                                                             decoder_cell.zero_state(dtype=tf.float32, batch_size=batch_size), 
                                                             output_layer)
        predicting_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                                                        impute_finished=True,
                                                                        maximum_iterations=limit_target_seq_len)
    ######## Model output tensor ########
    training_logits = tf.identity(training_decoder_output.rnn_output, name='trainings')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id, name='predictions')
    return training_logits, predicting_logits, max_target_seq_len
    
def build_optimizer(training_logits, predicting_logits, targets, target_seq_len, max_target_seq_len):
    ######## Optimization ########
    if is_training is not None:
        with tf.name_scope('optimization'):
            masks = tf.sequence_mask(target_seq_len, max_target_seq_len, dtype=tf.float32)
            # Loss function
            cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks, name='sequence_cost')
            # Add regularization to Loss
            regularization_loss = tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
            cost = cost + regularization_loss
            # Optimizer
            optimizer = tf.train.AdamOptimizer(lr)
            # Gradient Clipping
            gradients, vriables = zip(*optimizer.compute_gradients(cost))
            gradients = [None if gradient is None else tf.clip_by_norm(gradient, clip_norm) for gradient in gradients]
            train_step = optimizer.apply_gradients(zip(gradients, vriables), name='gradients')
    return cost, train_step

In [57]:
import os
import time
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.contrib import layers

################
# Set parameter
################
rnn_size = 150
num_layers = 3
slot_size = 172
embedding_size = 300
vocab_size = words_limit
limit_generation_len = max_title_length + 2
lr = 1e-3

lstm_dropout = 0.5
l2_reg_constant = 0.01
clip_norm = 50.0

patience = 0
num_epoch = 30
batch_size = 64
test_batch_size = 64

build_new_model = 1
if build_new_model:
    str_epoch = 0
    model_name = 'PTT_Encoder-Decoder_model_fixed_' + str(total_start_t)
else:
    str_epoch = 8
    model_name = 'PTT_Encoder-Decoder_model_fixed_1522379168.903185'

################
# Prepare to run session
################
num_instances = len(y_train)
iter_per_epoch = int(num_instances / batch_size)
iter_pct10 = int(iter_per_epoch / 10)
if (num_instances % batch_size) > 0:
    iter_per_epoch += 1
batch_cutoff = [0]
for i in range(iter_per_epoch - 1):
    batch_cutoff.append(batch_size * (i+1))
batch_cutoff.append(num_instances)

test_num_instances = len(y_test)
test_iter_per_epoch = int(test_num_instances / test_batch_size)

total_start_t = time.time()
best_validation_loss = 0.0
early_stop_counter = 0

mdl_dir = os.path.join(base_dir, 'model')
if not os.path.exists(mdl_dir):
    os.makedirs(mdl_dir)
model_dir = os.path.join(mdl_dir, model_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

################
# Run model session
################
train_graph = tf.Graph()
if build_new_model:
    ######## Build model parameters ########
    with train_graph.as_default():
        # Build input parameters
        is_training, inputs, targets, input_seq_len, target_seq_len, limit_target_seq_len = build_model_in()
        # Build model and  weight parameters
        training_logits, predicting_logits, max_target_seq_len = build_model(is_training, inputs, targets, 
                                                                             input_seq_len, target_seq_len, 
                                                                             limit_target_seq_len)
        # Build optimizer
        cost, train_step = build_optimizer(training_logits, predicting_logits,
                                           targets, target_seq_len, max_target_seq_len)

with tf.Session(graph=train_graph) as sess:
    if build_new_model:
        sess.run(tf.global_variables_initializer())
        best_epoch = 1
    else:
        ######## Load model parameters ########
        # Load model and weight parameters
        best_epoch = str_epoch
        load_model_dir = os.path.join(model_dir, 'model-{}'.format(str_epoch))
        loader = tf.train.import_meta_graph(load_model_dir + '.meta')
        loader.restore(sess, load_model_dir)
        # Load named input parameters
        is_training = train_graph.get_tensor_by_name('is_training:0')
        inputs = train_graph.get_tensor_by_name('inputs:0')
        targets = train_graph.get_tensor_by_name('targets:0')
        input_seq_len = train_graph.get_tensor_by_name('input_seq_len:0')
        target_seq_len = train_graph.get_tensor_by_name('target_seq_len:0')
        limit_target_seq_len = train_graph.get_tensor_by_name('limit_target_seq_len:0')
        # Load named output and optimizer parameters
        training_logits = train_graph.get_tensor_by_name('trainings:0')
        predicting_logits = train_graph.get_tensor_by_name('predictions:0')
        #cost = train_graph.get_tensor_by_name('optimization/sequence_cost:0')
        cost = train_graph.get_tensor_by_name('optimization/sequence_loss/Const:1')
        gradients = train_graph.get_tensor_by_name('optimization/gradients:0')
        
    ######## Run model epoch ########
    for e in range(str_epoch,str_epoch+num_epoch):
        rand_idxs = np.random.permutation(num_instances)
        print('\n========')
        print('epoch' + str(e+1))
        print('========')
        start_t = time.time()
        
        ######## Get random input batch and training model. ########
        for i in range(iter_per_epoch):
            X_batch = []
            Y_batch = []
            X_size = []
            Y_size = []
            for n in range(batch_cutoff[i],batch_cutoff[i+1]):
                X_batch.append(x_train[rand_idxs[n]])
                Y_batch.append(y_train[rand_idxs[n]])
                X_size.append(x_train_size[rand_idxs[n]])
                Y_size.append(y_train_size[rand_idxs[n]])
            max_X_size = max(X_size)
            max_Y_size = max(Y_size)
            X_batch = np.array(X_batch)
            Y_batch = np.array(Y_batch)
            X_size = np.array(X_size)
            Y_size = np.array(Y_size)
            
            _, loss = sess.run( [train_step, cost],
                                {is_training: True,
                                 inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size],
                                 input_seq_len: X_size, target_seq_len: Y_size,
                                 limit_target_seq_len: limit_generation_len})
            
            if i % iter_pct10 == iter_pct10 - 1:
                print('Iteration ',i+1,' (',round(i*100/iter_per_epoch,2),'%) Train Loss: ',
                      loss,'\t\t',end='\n')
        
        print('Train Loss: ',loss)
        
        ######## Get validation input batch and validating model. ########
        test_loss = []
        for i in range(test_iter_per_epoch):
            i_str = i * test_batch_size
            i_end = i_str + test_batch_size
            
            X_batch = np.array(x_test[i_str:i_end])
            Y_batch = np.array(y_test[i_str:i_end])
            X_size = np.array(x_test_size[i_str:i_end])
            Y_size = np.array(y_test_size[i_str:i_end])
            max_X_size = max(X_size)
            max_Y_size = max(Y_size)
            y_loss = sess.run( cost,{inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size],
                                     input_seq_len: X_size, target_seq_len: Y_size,
                                     limit_target_seq_len: limit_generation_len})
            test_loss.append(y_loss)
        else:
            test_loss = np.asarray(test_loss)
            validation_loss = test_loss.mean()
            validation_loss_std = test_loss.std()
        print('Test loss: ',validation_loss,'; Std: ',validation_loss_std)
        
        ######## Generate title form testing content and validating model. ########
        answer_logits = sess.run( predicting_logits, 
                                {inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size], 
                                 input_seq_len: X_size, target_seq_len: Y_size,
                                 limit_target_seq_len: limit_generation_len})
        output_index = random.randint(0,test_batch_size-1)
        print('Target Words: {}'.format(" ".join([anti_vocab[i] for i in Y_batch[output_index] if i != 0])))
        print('Response Words: {}'.format(" ".join([anti_vocab[i] for i in answer_logits[output_index] if i != 0])))
        
        ######## Save model weight parameters and information. ########
        if validation_loss < best_validation_loss or e == 0:
            best_validation_loss = validation_loss
            best_epoch = e
            early_stop_counter = 0
            print('Save best score!! '+str(best_validation_loss))
        else:
            early_stop_counter += 1
            print('Count early stop!! '+str(early_stop_counter))

        print('Elapsed time in epoch ' + str(e+1) + ': ' + str(time.time() - start_t) + ' [s]')

        model_path = os.path.join(model_dir, 'model-%d' %(e+1))
        saver = tf.train.Saver()
        saver.save(sess, model_path)
        
        if patience != 0 and early_stop_counter >= patience:
            break

print('\n#######')
print('Best model')
print('#######')
print('Stop by early stopping')
print('Best score: ', best_validation_loss, 'Beat model: ', best_epoch)
print('Elapsed time in total: ' + str(time.time() - total_start_t))


epoch1
Iteration  108  ( 9.91 %) Train Loss:  7.15027 		
Iteration  216  ( 19.91 %) Train Loss:  6.26867 		
Iteration  324  ( 29.91 %) Train Loss:  5.85552 		
Iteration  432  ( 39.91 %) Train Loss:  5.81335 		
Iteration  540  ( 49.91 %) Train Loss:  5.36729 		
Iteration  648  ( 59.91 %) Train Loss:  5.28365 		
Iteration  756  ( 69.91 %) Train Loss:  4.96648 		
Iteration  864  ( 79.91 %) Train Loss:  5.38616 		
Iteration  972  ( 89.91 %) Train Loss:  5.24372 		
Iteration  1080  ( 99.91 %) Train Loss:  5.41377 		
Train Loss:  5.41377
Test loss:  5.11273 ; Std:  0.145225
Target Words: <GO> 七 產業 將 鬆綁 陸資 控制力 <EOS>
Response Words: <GO> 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎 嗎
Save best score!! 5.11273
Elapsed time in epoch 1: 1191.3176193237305 [s]

epoch2
Iteration  108  ( 9.91 %) Train Loss:  5.17462 		
Iteration  216  ( 19.91 %) Train Loss:  5.30611 		
Iteration  324  ( 29.91 %) Train Loss:  5.1483 		
Iteration  432  ( 39.91 %) Train Loss:  4.9981 		
Iteration  540  ( 49.91 %) Train Los

Iteration  972  ( 89.91 %) Train Loss:  3.7756 		
Iteration  1080  ( 99.91 %) Train Loss:  4.0545 		
Train Loss:  4.0545
Test loss:  4.05704 ; Std:  0.157118
Target Words: <GO> 西斯 好球 帶 <EOS>
Response Words: <GO> 包子 之靈 八卦 ら ら ら <EOS>
Save best score!! 4.05704
Elapsed time in epoch 11: 1162.8306076526642 [s]

epoch12
Iteration  108  ( 9.91 %) Train Loss:  3.85735 		
Iteration  216  ( 19.91 %) Train Loss:  3.81178 		
Iteration  324  ( 29.91 %) Train Loss:  3.93788 		
Iteration  432  ( 39.91 %) Train Loss:  3.7911 		
Iteration  540  ( 49.91 %) Train Loss:  4.11452 		
Iteration  648  ( 59.91 %) Train Loss:  3.94636 		
Iteration  756  ( 69.91 %) Train Loss:  3.79176 		
Iteration  864  ( 79.91 %) Train Loss:  3.93067 		
Iteration  972  ( 89.91 %) Train Loss:  3.81525 		
Iteration  1080  ( 99.91 %) Train Loss:  3.86982 		
Train Loss:  3.86982
Test loss:  4.02646 ; Std:  0.159897
Target Words: <GO> 元 的 結尾 <EOS>
Response Words: <GO> 研發 伴 時 伴 研發 伴 伴 伴 伴 伴 伴 伴 研發 伴 伴 伴 伴 伴 伴 伴 伴
Save best score!! 

Iteration  648  ( 59.91 %) Train Loss:  3.8373 		
Iteration  756  ( 69.91 %) Train Loss:  3.72254 		
Iteration  864  ( 79.91 %) Train Loss:  3.62509 		
Iteration  972  ( 89.91 %) Train Loss:  3.58418 		
Iteration  1080  ( 99.91 %) Train Loss:  3.63217 		
Train Loss:  3.63217
Test loss:  3.83875 ; Std:  0.172689
Target Words: <GO> 七 產業 將 鬆綁 陸資 控制力 <EOS>
Response Words: <GO> 列車 製程 製程 <EOS>
Save best score!! 3.83875
Elapsed time in epoch 22: 1160.7914729118347 [s]

epoch23
Iteration  108  ( 9.91 %) Train Loss:  3.53917 		
Iteration  216  ( 19.91 %) Train Loss:  3.72626 		
Iteration  324  ( 29.91 %) Train Loss:  3.58434 		
Iteration  432  ( 39.91 %) Train Loss:  3.63087 		
Iteration  540  ( 49.91 %) Train Loss:  3.61985 		
Iteration  648  ( 59.91 %) Train Loss:  3.69169 		
Iteration  756  ( 69.91 %) Train Loss:  3.63335 		
Iteration  864  ( 79.91 %) Train Loss:  3.42341 		
Iteration  972  ( 89.91 %) Train Loss:  3.95316 		
Iteration  1080  ( 99.91 %) Train Loss:  3.67971 		
Train Loss:  3.

In [74]:
import os
import random
################################
#Generate title form testing content and validating model. 
################################
base_dir = 'C:/Users/User/Raw data/PTT'
model_name = 'PTT_Encoder-Decoder_model_fixed_1522421878.8546176'
epoch = 20
model_path = os.path.join(base_dir, 'model', model_name, 'model-{}'.format(epoch))
print('Test epoch: ', epoch)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load model
    loader = tf.train.import_meta_graph(model_path + '.meta')
    loader.restore(sess, model_path)
    # Named parameters
    #is_training = loaded_graph.get_tensor_by_name('is_training:0')
    inputs = loaded_graph.get_tensor_by_name('inputs:0')
    targets = loaded_graph.get_tensor_by_name('targets:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    input_seq_len = loaded_graph.get_tensor_by_name('input_seq_len:0')
    target_seq_len = loaded_graph.get_tensor_by_name('target_seq_len:0')
    limit_target_seq_len = loaded_graph.get_tensor_by_name('limit_target_seq_len:0')
    
    for _ in range(8):
        i_str = random.randint(0,test_num_instances-1)
        i_end = i_str + 1

        X_batch = np.tile(x_test[i_str], (test_batch_size, 1))
        Y_batch = np.tile(y_test[i_str], (test_batch_size, 1))
        X_size = np.tile(x_test_size[i_str], (test_batch_size))
        Y_size = np.tile(y_test_size[i_str], (test_batch_size))
        max_X_size = x_test_size[i_str]
        max_Y_size = y_test_size[i_str]
        answer_logits = sess.run( logits, {inputs: X_batch, targets: Y_batch, 
                                             input_seq_len: X_size, target_seq_len: Y_size,
                                             limit_target_seq_len: limit_generation_len})

        output_index = 0 #random.randint(0,test_batch_size-1)
        print('\n========')
        print('Test index: ', i_str + output_index)
        print('========')
        print('Test input Words: {}'.format(" ".join([anti_vocab[i] for i in X_batch[output_index] if i != 0])))
        print('Target Words: {}'.format(" ".join([anti_vocab[i] for i in Y_batch[output_index] if i != 0])))
        print('Response Words: {}'.format(" ".join([anti_vocab[i] for i in answer_logits[output_index] if i != 0])))

Test epoch:  20
INFO:tensorflow:Restoring parameters from C:/Users/User/Raw data/PTT\model\PTT_Encoder-Decoder_model_fixed_1522421878.8546176\model-20

Test index:  7005
Test input Words: <GO> 我 想 徵 長髮 女生 其他 的 條件 一律 不 限定 只要 是 女生 而且 頭髮 曾經 很長 很長 可 就 可以 了 不管 你 是 靚女 宅女 還是 玉女 羅莉 御姐 熟女 都 ok 不是 正妹 也 沒關係 只要 你 頭髮 曾經 有 她 一半 就 可以 喔 妳是 長髮 過腰 的 女生 嗎 妳 想 把 長髮 剪短 嗎 我 想 收集 長髮 喔 讓 我們 先 聊聊 唄 <EOS>
Target Words: <GO> 超 長髮 女生 <EOS>
Response Words: <GO> 一些 機構 鐵血 心得 <EOS>

Test index:  4390
Test input Words: <GO> 台積 薄膜 太陽能 模組 轉換 效率 14 2 字 字 自由 時報 2012 年 9 月 6 日 上午 4 24 〔 自由 時報 記者 洪友芳 新竹 報導 〕 台積電 2330 轉投資 的 台積 太陽能 公司 的 薄膜 太陽能 模組 生產 效率 有 突破性 進展 轉換 效率 達 14 2 預計 明年初 取得 國際 認證 不過 因 目前 太陽能 市況 差 台積電 估計 台積 太 公司 還難 獲利 台積 太陽能 公司 指出 位於 台中 的 先進 廠房 研發 試產 線 近期 成功 生產 出 14 2 高 轉換 效率 的 薄膜 太陽能 模組 模組 已 進入 UL 及 的 認證 程序 預計 將在 2013 年 初 獲得 認證 目前 國內 太陽能 電池 相關 業者 以 生產 太陽能 電池 為主 台積 太陽能 市場 區隔 專注 研 生產 薄膜 太陽能 模組 領域 此 較 高 比起 轉換 效率 達 17 18 薄膜 太 模組 14 2 業界 不錯 台積電 表示 台積 太陽能 產品 已 小量 出貨 因 目前 太陽能 市況 差 不 打算 大量 產出 營運 也 還在 虧損 中 不景氣 階段 公司 計畫 加強 研發